In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Projects\\End-to-End_Pricing'

In [3]:
from pathlib import Path

In [4]:
from dataclasses import dataclass
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    mlflow_uri: str

In [5]:
from src.house_reg._utils.common import read_yaml,create_directories
from src.house_reg.constant import CONFIG_PATH,PARAMS_PATH

[2024-12-27 03:20:23,288: INFO: __init__: Logging setup successful.]


In [6]:
class ConfigurationManager():
    def __init__(
        self,
        config_file_path= CONFIG_PATH,
        params_file_path= PARAMS_PATH):

        self.config= read_yaml(config_file_path)
        self.params= read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self)-> EvaluationConfig:

        eval_config=EvaluationConfig(
            path_of_model="artifacts\prepare_base_model\model.pkl",
            training_data="artifacts\data_ingestion\Housing.csv",
            mlflow_uri="https://dagshub.com/harsh9769/End-to-End_Price.mlflow")
        
        return eval_config

In [9]:
import pickle
from pathlib import Path
import mlflow
import dagshub
from urllib.parse import urlparse
import json


class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config = config

    @staticmethod
    def load_model(path):
        """Load a model from the given path."""
        with open(str(path), 'rb') as f:
            model = pickle.load(f)
        return model

    @staticmethod
    def load_data():
        """Load X_test and y_test from pre-saved files."""
        with open('artifacts/prepare_base_model/X_test.pkl', 'rb') as f:
            X_test = pickle.load(f)
        with open('artifacts/prepare_base_model/y_test.pkl', 'rb') as f:
            y_test = pickle.load(f)
        return X_test, y_test

    def evaluation(self):
        """Evaluate the model on the test data and save the score."""
        self.model = self.load_model(r"artifacts\prepare_base_model\model.pkl")

        X_test, y_test = self.load_data()
        self.score = self.model.score(X_test, y_test)
        self.save_score()

    def save_score(self):
        """Save the evaluation score to a JSON file."""
        scores = {"score": self.score}
        with open("scores.json", "w") as f:
            json.dump(scores, f)

    def log_into_mlflow(self):
        """Log parameters, metrics, and the model into MLflow, using DagsHub integration."""
        # Initialize DagsHub integration
        dagshub.init(
            repo_owner='harsh9769',
            repo_name='End-to-End_Price',
            mlflow=True
        )

        # Start an MLflow run and log parameters and metrics
        with mlflow.start_run():
            mlflow.log_param('model_path', str(self.config.path_of_model))
            mlflow.log_metric('score', self.score)



In [10]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-12-27 03:22:16,150: INFO: common: yaml file:config\config.yaml loaded successfully]
[2024-12-27 03:22:16,153: INFO: common: yaml file:params.yaml loaded successfully]
[2024-12-27 03:22:16,156: INFO: common: created directory:artifacts]
[2024-12-27 03:22:16,667: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/harsh9769/End-to-End_Price "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "harsh9769/End-to-End_Price"

[2024-12-27 03:22:16,678: INFO: helpers: Initialized MLflow to track repo "harsh9769/End-to-End_Price"]


Repository harsh9769/End-to-End_Price initialized!

[2024-12-27 03:22:16,681: INFO: helpers: Repository harsh9769/End-to-End_Price initialized!]
